# Install dependencies

In [ ]:
!pip install -q TTS
!sudo apt-get -y install espeak-ng

## Insert Dataset

You can use kaggle api to download dataset:
https://www.kaggle.com/datasets/magnoliasis/persian-tts-dataset-famale

In [8]:
!mkdir train_output

You should set your own dataset path in bellow cell like this:

exp. : my dataset path is `/contents/persian-tts-dataset-famale` and these are files under folder:
```
/contents/persian-tts-dataset-famale
|-- wavs
|   |-- 1.wav
|   |-- 2.wav
|   |-- 3.wav
|   |-- ...
|
|
|-- metadata.csv
```
So this part of code should be (lines 26-28 in train_glowtts.py or bellow cell) like this:
```
dataset_config = BaseDatasetConfig(
    formatter="mozilla", meta_file_train="metadata.csv", path="/contents/persian-tts-dataset-famale" 
)
```


In [31]:
code='''import os

# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig , CharactersConfig
from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

# we use the same path as this script as our training folder.
output_path = os.path.dirname(os.path.abspath(__file__))

# DEFINE DATASET CONFIG
# Set LJSpeech as our target dataset and define its path.
# You can also use a simple Dict to define the dataset and pass it to your custom formatter.


dataset_config = BaseDatasetConfig(
    formatter="mozilla", meta_file_train="metadata.csv", path="/kaggle/input/persian-tts-dataset-famale" 
)

audio_config = BaseAudioConfig(
    sample_rate=24000,
    do_trim_silence=True,
    resample=False
    
)

character_config=CharactersConfig(
  characters='ءابتثجحخدذرزسشصضطظعغفقلمنهويِپچژکگیآأؤإئًَُّ',
  punctuations='!(),-.:;? ̠،؛؟‌<>',
  phonemes='ˈˌːˑpbtdʈɖcɟkɡqɢʔɴŋɲɳnɱmʙrʀⱱɾɽɸβfvθðszʃʒʂʐçʝxɣχʁħʕhɦɬɮʋɹɻjɰlɭʎʟaegiouwy',
  pad="<PAD>",
  eos="<EOS>",
  bos="<BOS>",
  blank="<BLNK>",
  characters_class="TTS.tts.utils.text.characters.IPAPhonemes",
  )
# INITIALIZE THE TRAINING CONFIGURATION
# Configure the model. Every config class inherits the BaseTTSConfig.
config = GlowTTSConfig(
    batch_size=8,#batch_size=32,
    eval_batch_size=4,#eval_batch_size=16,
    num_loader_workers=0,
    num_eval_loader_workers=0,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    save_step=1000,
    text_cleaner="basic_cleaners",
    use_phonemes=True,
    phoneme_language="fa",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    characters=character_config,
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    audio=audio_config,
    test_sentences=[
        "سلطان محمود در زمستانی سخت به طلخک گفت که: با این جامه ی یک لا در این سرما چه می کنی ",
        "مردی نزد بقالی آمد و گفت پیاز هم ده تا دهان بدان خو شبوی سازم.",
        "از مال خود پاره ای گوشت بستان و زیره بایی معطّر بساز",
        "یک بار هم از جهنم بگویید.",
        "یکی اسبی به عاریت خواست"
    ],

    
)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# If characters are not defined in the config, default characters are passed to the config
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# INITIALIZE THE MODEL
# Models take a config object and a speaker manager as input
# Config defines the details of the model like the number of layers, the size of the embedding, etc.
# Speaker manager is used by multi-speaker models.
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

# INITIALIZE THE TRAINER
# Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training,
# distributed training, etc.
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

# AND... 3,2,1... 🚀
trainer.fit()
'''
f=open("train_output/train_glowtts.py","w",encoding="utf-8")

f.write(code)

f.close()

# Start training

For training on first time run this cell: 

In [ ]:
!PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:512" python train_output/train_glowtts.py

to continue training run this cell: 
- set your own `continue_path`

In [ ]:
!PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:512" python train_output/train_glowtts.py --continue_path 'train_output/run-December-23-2022_02+09PM-0000000'

# finetuning

In [ ]:
!mkdir train_output/my_model

Download my last pretrained checkpoint and `config.json` :

In [ ]:
!wget "https://huggingface.co/Kamtera/persian-tts-female-glow_tts/resolve/main/best_model.pth" -O "train_output/my_model/best_model.pth"
!wget "https://huggingface.co/Kamtera/persian-tts-female-glow_tts/resolve/main/config.json" -O "train_output/my_model/config.json"

You could use default settings in `config.json` or edit it with your own parameters.

Also , you can set parameters from  cli args like this:

In [ ]:
!python train_output/train_glowtts.py \
--restore_path "train_output/my_model/best_model.pth" \
--coqpit.run_name "glow-tts-finetune" \
--coqpit.lr 0.00001